In [57]:
in_colab = 'google.colab' in str(get_ipython())
if in_colab == True:
  !pip install yfinance
  import yfinance as yf
  import numpy as np
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.metrics import accuracy_score
  import matplotlib.pyplot as plt
  from sklearn.kernel_ridge import KernelRidge
else:
  import yfinance as yf
  import numpy as np
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from sklearn.metrics import accuracy_score
  import matplotlib.pyplot as plt
  from sklearn.kernel_ridge import KernelRidge

In [233]:
crypto_btc = yf.download(tickers='BTC-USD', start='2018-01-01', end='2022-12-31').drop(columns=['Adj Close','High', 'Low']).round(2).reset_index()
crypto_eth = yf.download(tickers='ETH-USD', start='2018-01-01', end='2022-12-31').drop(columns=['Adj Close']).round(4).reset_index()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [70]:
crypto_eth

,Date,Open,Close,Volume
0,2018-01-01 00:00:00+00:00,755.7570,772.6410,2595760128
1,2018-01-02 00:00:00+00:00,772.3460,884.4440,5783349760
2,2018-01-03 00:00:00+00:00,886.0000,962.7200,5093159936
3,2018-01-04 00:00:00+00:00,961.7130,980.9220,6502859776
4,2018-01-05 00:00:00+00:00,975.7500,997.7200,6683149824
...,...,...,...,...
1820,2022-12-26 00:00:00+00:00,1218.9202,1226.9744,3282098400
1821,2022-12-27 00:00:00+00:00,1226.9871,1212.7916,4091530737
1822,2022-12-28 00:00:00+00:00,1212.7366,1189.9861,4991669631
1823,2022-12-29 00:00:00+00:00,1190.0101,1201.5953,4132233940


In [13]:
#crypto_btc['Percent Change'] = crypto_btc['Close'].pct_change()
#crypto_eth['Percent Change'] = crypto_eth['Close'].pct_change()

In [ ]:
crypto_btc.plot(
    x = 'Date',
    y = 'Close',
    grid = True,
    figsize = (20,10)
)

In [60]:
crypto_btc.head()

,Date,Open,Close,Volume
0,2018-01-01 00:00:00+00:00,14112.2,13657.2,10291200000
1,2018-01-02 00:00:00+00:00,13625.0,14982.1,16846600192
2,2018-01-03 00:00:00+00:00,14978.2,15201.0,16871900160
3,2018-01-04 00:00:00+00:00,15270.7,15599.2,21783199744
4,2018-01-05 00:00:00+00:00,15477.2,17429.5,23840899072


In [265]:
df = ()

def kernel_predict(df_pred):
    X = df.drop(columns=['Date'])
    y = df[['Close']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=42)
    if df is crypto_btc:
        kr = KernelRidge(kernel='linear', alpha=5000)
    else:
        kr = KernelRidge(kernel='polynomial', degree=1, gamma=.05)
    kr.fit(X_train, y_train)
    predict = kr.predict(X)
    if df is crypto_btc:
        df_pred = df.copy().drop(columns=['Open','Volume']).rename(columns={'Close':'Actual Price'})
    else:
        df_pred = df.copy().drop(columns=['Open']).rename(columns={'Close':'Actual Price'})
    df_pred['Predicted Price'] = predict
    df_pred['Price Accuracy']= (df_pred['Predicted Price']/df_pred['Actual Price']-1)*100
    return df_pred


In [236]:
df = crypto_btc
btc_predict=pd.DataFrame(kernel_predict(df))
btc_predict['Price Accuracy'].describe()

/Users/playb3yond40gb/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "


count    1825.000000
mean        0.302494
std         0.945725
min        -3.863393
25%        -0.127481
50%         0.249405
75%         0.687104
max        13.623790
Name: Price Accuracy, dtype: float64

In [266]:
df = crypto_eth
eth_predict=pd.DataFrame(kernel_predict(df))
eth_predict['Price Accuracy'].describe()

/Users/playb3yond40gb/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:196: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  "Singular matrix in solving dual problem. Using "


count    1825.000000
mean       -0.524432
std         2.650934
min       -12.233731
25%        -1.977697
50%        -0.582229
75%         0.668506
max        31.456442
Name: Price Accuracy, dtype: float64

In [256]:
eth_predict

,Date,High,Low,Actual Price,Volume,Predicted Price,Price Accuracy
0,2018-01-01 00:00:00+00:00,782.5300,742.0040,772.6410,2595760128,761.9375,-1.385313
1,2018-01-02 00:00:00+00:00,914.8300,772.3460,884.4440,5783349760,834.1250,-5.689337
2,2018-01-03 00:00:00+00:00,974.4710,868.4510,962.7200,5093159936,920.8125,-4.353031
3,2018-01-04 00:00:00+00:00,1045.0800,946.0860,980.9220,6502859776,982.1250,0.122640
4,2018-01-05 00:00:00+00:00,1075.3900,956.3250,997.7200,6683149824,997.4375,-0.028315
...,...,...,...,...,...,...,...
1820,2022-12-26 00:00:00+00:00,1226.9744,1214.3359,1226.9744,3282098400,1220.1875,-0.553141
1821,2022-12-27 00:00:00+00:00,1230.4181,1205.8956,1212.7916,4091530737,1216.8125,0.331541
1822,2022-12-28 00:00:00+00:00,1213.1289,1185.7021,1189.9861,4991669631,1198.5000,0.715462
1823,2022-12-29 00:00:00+00:00,1204.1416,1188.3602,1201.5953,4132233940,1194.0625,-0.626900


In [ ]:
btc_predict.plot(
    x='Date',
    y=['Actual Price','Predicted Price' ],
    grid = True,
    figsize=(20,10)
)